In [1]:
# ================================
# پیش‌بینی 7 روز آینده با RandomForest برای 4 سیگنال روغن‌کاری
# با استفاده از past covariates (فیچرهای دیگر)
# ================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

plt.rcParams['figure.figsize'] = (16, 10)
sns.set_style("whitegrid")

# ================================
# مسیرها
# ================================
INPUT_FILE = r"C:\BI\lube_oil_system_data_g11.xlsx"
OUTPUT_EXCEL = r"C:\BI\forecast_lube_oil_params_RF_with_cov.xlsx"
OUTPUT_PLOTS = r"C:\BI\plots_timeseries_rf_with_cov\\"
LOG_FILE = r"C:\BI\regression_assetid_rf_with_cov_log.txt"
os.makedirs(r"C:\BI", exist_ok=True)
os.makedirs(OUTPUT_PLOTS, exist_ok=True)

def log(msg):
    print(msg)
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}\n")

log("شروع اجرای مدل RandomForest با past covariates برای 4 سیگنال و پیش‌بینی 7 روز آینده")

# ================================
# خواندن داده و ساخت datetime
# ================================
df = pd.read_excel(INPUT_FILE)
log(f"داده خوانده شد → {len(df):,} ردیف")

df["datetime"] = pd.to_datetime(df["RecordDate"] + " " + df["RecordTime"], errors="coerce")
df = df.sort_values("datetime").reset_index(drop=True)

# ================================
# نام‌های خوانا
# ================================
friendly_names = {
    "AssetID_8341": "Lube oil filter diffrential pressure",
    "AssetID_8342": "Lube oil tank level",
    "AssetID_8343": "Lube oil tank temperature",
    "AssetID_8344": "Lube oil tank vapor pressure",
    "AssetID_8346": "Over speed pressure",
    "AssetID_9286": "Main lube oil pump press",
    "AssetID_9287": "Lube oil line pressure",
}
df.rename(columns=friendly_names, inplace=True)

# ================================
# سیگنال‌های هدف
# ================================
target_cols = [
    "Main lube oil pump press",
    "Lube oil line pressure",
    "Lube oil tank temperature",
    "Lube oil tank level"
]

SEQ_LEN = 30
HORIZON = 7

# ================================
# آماده‌سازی داده‌های روزانه چندمتغیره
# ================================
df_multi = df.dropna(subset=target_cols).set_index("datetime")[target_cols]
daily_multi = df_multi.resample("D").mean().interpolate()

if len(daily_multi) < 60:
    raise ValueError("طول سری روزانه کمتر از 60 است؛ برای آموزش پایدار داده کافی نیست.")

log(f"سری چندمتغیره: {len(daily_multi)} روز از {daily_multi.index.min().date()} تا {daily_multi.index.max().date()}")

# ================================
# اجرای مدل برای هر سیگنال با past covariates
# ================================
with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
    for target_col in target_cols:
        log(f"شروع پردازش سیگنال: {target_col}")

        # نرمال‌سازی همه فیچرها
        scalers = {col: MinMaxScaler() for col in target_cols}
        scaled_cols = [scalers[col].fit_transform(daily_multi[[col]]) for col in target_cols]
        scaled_array = np.concatenate(scaled_cols, axis=1)

        # ساخت داده‌های آموزشی
        X, Y = [], []
        for i in range(len(scaled_array) - SEQ_LEN - HORIZON + 1):
            window = scaled_array[i:i+SEQ_LEN]  # شامل همه فیچرها
            X.append(window.flatten())
            target_seq = scaled_array[i+SEQ_LEN:i+SEQ_LEN+HORIZON, target_cols.index(target_col)]
            Y.append(target_seq)
        X, Y = np.array(X), np.array(Y)

        log(f"آموزش مدل RandomForest برای '{target_col}' | نمونه‌ها: {X.shape[0]} | ورودی: {X.shape[1]} ویژگی")

        model = RandomForestRegressor(
            n_estimators=300,
            max_depth=12,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X, Y)

        # پیش‌بینی 7 روز آینده
        last_seq = scaled_array[-SEQ_LEN:].flatten().reshape(1, -1)
        pred_scaled = model.predict(last_seq)[0]
        pred_values = scalers[target_col].inverse_transform(pred_scaled.reshape(-1, 1)).flatten()

        future_dates = pd.date_range(start=daily_multi.index[-1] + pd.Timedelta(days=1), periods=HORIZON, freq="D")
        forecast_df = pd.DataFrame({
            "datetime": future_dates,
            f"{target_col} forecast": pred_values
        })

        # ذخیره در اکسل
        daily_multi[[target_col]].reset_index().to_excel(writer, sheet_name=f"{target_col}_History", index=False)
        forecast_df.to_excel(writer, sheet_name=f"{target_col}_Forecast_RF_with_cov", index=False)

        # رسم نمودار
        plt.figure(figsize=(16, 8))
        plt.plot(daily_multi.index, daily_multi[target_col], label="واقعی (روزانه)", color="steelblue")
        plt.plot(forecast_df["datetime"], forecast_df[f"{target_col} forecast"], label="پیش‌بینی 7 روز آینده (RF با covariates)", color="green", linewidth=2)
        plt.title(f"{target_col} - پیش‌بینی 7 روز آینده با ورودی 1 ماه اخیر (RandomForest با past covariates)")
        plt.xlabel("تاریخ")
        plt.ylabel("مقدار")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_PLOTS, f"rf_with_cov_7day_{target_col.replace(' ', '_')}.png"), dpi=200, bbox_inches="tight")
        plt.close()

log("تمام شد! خروجی RandomForest با past covariates برای 4 سیگنال ذخیره شد")
print("\n" + "="*80)
print("RandomForest forecasts with covariates ساخته شد و ذخیره شد")
print(f"فایل اکسل: {OUTPUT_EXCEL}")
print(f"نمودارها: {OUTPUT_PLOTS}")
print("="*80)


شروع اجرای مدل RandomForest با past covariates برای 4 سیگنال و پیش‌بینی 7 روز آینده
داده خوانده شد → 10,927 ردیف
سری چندمتغیره: 1690 روز از 2021-04-03 تا 2025-11-17
شروع پردازش سیگنال: Main lube oil pump press
آموزش مدل RandomForest برای 'Main lube oil pump press' | نمونه‌ها: 1654 | ورودی: 120 ویژگی
شروع پردازش سیگنال: Lube oil line pressure
آموزش مدل RandomForest برای 'Lube oil line pressure' | نمونه‌ها: 1654 | ورودی: 120 ویژگی
شروع پردازش سیگنال: Lube oil tank temperature
آموزش مدل RandomForest برای 'Lube oil tank temperature' | نمونه‌ها: 1654 | ورودی: 120 ویژگی
شروع پردازش سیگنال: Lube oil tank level
آموزش مدل RandomForest برای 'Lube oil tank level' | نمونه‌ها: 1654 | ورودی: 120 ویژگی
تمام شد! خروجی RandomForest با past covariates برای 4 سیگنال ذخیره شد

RandomForest forecasts with covariates ساخته شد و ذخیره شد
فایل اکسل: C:\BI\forecast_lube_oil_params_RF_with_cov.xlsx
نمودارها: C:\BI\plots_timeseries_rf_with_cov\\
